# 09 Case Study

0. Introduction
1. Selecting Topics
    1. Choosing four Topics
    2. Process of Selecting Tweets 
    3. Presenting overview
    4. Topic 1
    5. Topic 2
    6. Topic 3
    7. Topic 4
2. LDA
3. BerTopic

In [10]:
from src.SampleTranslation05.translation_01 import load_samples
import pandas as pd
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2

In [12]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [92]:
df_1 = pd.read_excel("samples.xlsx")

In [89]:
df_1 = df_1[~df_1['cleaned_text_translated'].isna()]

In [91]:
df_1[df_1['cleaned_text_translated'].str.contains('killed')]['cleaned_text_translated']

115      there can actually only be one comprehensible ...
156      the interior ministry of  ukraine reported on ...
274      according to the mayor of  mariupol, more than...
665      🇺🇦  ukraine  update 🔹ukrainian railway reports...
790      in the kiev region, residents shot a family: t...
                               ...                        
48609    euronews 🔵  ukraine: a little girl killed and ...
48666    ❗️5 civilians, residents of the belgorod regio...
48806    🇺🇦🇷🇺 a russian ural-4320 transport truck was c...
48856    ukraine at least three people killed in russia...
48871    (the   war in  ukraine, live: 487 children hav...
Name: cleaned_text_translated, Length: 504, dtype: object

In [51]:
df_1['topics'].unique()

array([nan, 'sanctions', 'unjustified_war', 'arms_delivery',
       'people_killed'], dtype=object)

In [93]:
df_1['topics'].value_counts()

topics
sanctions             26
unjustified_war       26
arms_delivery         26
people_killed         25
kriegsbeschreibung     3
santioncs              1
Name: count, dtype: int64

In [94]:
df = df_1[~df_1['topics'].isna()]

In this case study, i will hand select 100 tweets and divide them into three topics. Then i will run LDA, BertTopic and BertTopic Multilingual on them and evaluate them against my handpicked topics.

In [96]:
df.to_csv('/Users/robinfeldmann/TopicAnalysisRUWTweets/src/CaseStudy09/case_study.csv')